In [4]:
import xml.etree.ElementTree as ET
import requests
import psycopg2
import os
import pandas as pd
from sqlalchemy import create_engine
from dotenv import load_dotenv
from lxml import etree
import time

# Load environment variables (should be identical to other scripts)
load_dotenv()

# Define the database configurations for each environment
db_configs = [
    {
        'dbname': 'Antigone',
        'user': os.getenv('DB_USER', 'DB_USER'),
        'password': os.getenv('DB_PASSWORD', 'DB_PASSWORD'),
        'host': os.getenv('DB_HOST', 'DB_HOST'),
        'schema': 'Source'
    },
    {
        'dbname': 'Ismene',
        'user': os.getenv('DB_USER', 'DB_USER'),
        'password': os.getenv('DB_PASSWORD', 'DB_PASSWORD'),
        'host': os.getenv('DB_HOST', 'DB_HOST'),
        'schema': 'Source'
    },
    {
        'dbname': 'Eteocles',
        'user': os.getenv('DB_USER', 'DB_USER'),
        'password': os.getenv('DB_PASSWORD', 'DB_PASSWORD'),
        'host': os.getenv('DB_HOST', 'DB_HOST'),
        'schema': 'Source'
    },
    {
        'dbname': 'Polyneices',
        'user': os.getenv('DB_USER', 'DB_USER'),
        'password': os.getenv('DB_PASSWORD', 'DB_PASSWORD'),
        'host': os.getenv('DB_HOST', 'DB_HOST'),
        'schema': 'Source'
    }
]

# Function to fetch and parse XML data from Eurostat API
def fetch_eurostat_data(dataset_code):
    url = f"https://ec.europa.eu/eurostat/api/dissemination/sdmx/3.0/data/dataflow/ESTAT/{dataset_code}/1.0?compress=false"
    response = requests.get(url)
    if response.status_code != 200:
        print(f"Failed to fetch data for {dataset_code}. Status code: {response.status_code}")
        return None

    root = etree.fromstring(response.content)
    structured_data = []
    for series in root.findall('.//Series', namespaces={}):
        series_data = series.attrib
        for obs in series.findall('.//Obs', namespaces={}):
            obs_data = obs.attrib
            record = {**series_data, **obs_data}
            structured_data.append(record)

    df = pd.DataFrame(structured_data)
    if df.empty:
        print("DataFrame is empty, please check the XML structure or namespace.")
    else:
        print(df.head())

    return df

# Function to upload data to PostgreSQL
def upload_to_postgres(df, table_name, config):
    engine = create_engine(f'postgresql://{config["user"]}:{config["password"]}@{config["host"]}/{config["dbname"]}')
    df.columns = [col.replace(' ', '_').lower() for col in df.columns]
    df = df.apply(pd.to_numeric, errors='ignore')
    df.to_sql(table_name, engine, schema=config['schema'], if_exists='replace', index=False, chunksize=500)
    print(f"Data uploaded to {config['dbname']} successfully.")

# Dataset mapping to table names
dataset_mapping = {

    'demo_divcz': 'divorces_by_citizenship_of_wife_and_husband',
    'demo_divcb': 'divorces_by_country_of_birth_of_wife_and_husband',
    'demo_ndivind': 'divorce_indicators',
    'tps00014': 'mean_age_at_first_marriage_by_sex',
    'tps00216': 'crude_divorce_rate',
    'tps00206': 'crude_marriage_rate'

}

# Main function to orchestrate data ingestion
def main():
    start_time = time.time()

    for dataset_code, table_name in dataset_mapping.items():
        print(f"\nFetching dataset: {dataset_code}")
        data = fetch_eurostat_data(dataset_code)
        if data is not None:
            for config in db_configs:
                upload_to_postgres(data, table_name, config)

    # Calculate runtime
    end_time = time.time()
    runtime = end_time - start_time
    print(f"Total script runtime: {runtime} seconds")

if __name__ == "__main__":
    main()



Fetching dataset: demo_divcz
         citizen freq geo        partner unit TIME_PERIOD OBS_VALUE OBS_FLAG
0  EU27_2007_FOR    A  AT  EU27_2007_FOR   NR        2012       251      NaN
1  EU27_2007_FOR    A  BG  EU27_2007_FOR   NR        2012         0      NaN
2  EU27_2007_FOR    A  CH  EU27_2007_FOR   NR        2012       462      NaN
3  EU27_2007_FOR    A  CZ  EU27_2007_FOR   NR        2012        41      NaN
4  EU27_2007_FOR    A  DK  EU27_2007_FOR   NR        2012        95      NaN
Data uploaded to Antigone successfully.
Data uploaded to Ismene successfully.
Data uploaded to Eteocles successfully.
Data uploaded to Polyneices successfully.

Fetching dataset: demo_divcb
         c_birth freq geo        partner unit TIME_PERIOD OBS_VALUE
0  EU27_2007_FOR    A  DK  EU27_2007_FOR   NR        2012       110
1  EU27_2007_FOR    A  EE  EU27_2007_FOR   NR        2012         1
2  EU27_2007_FOR    A  FI  EU27_2007_FOR   NR        2012        95
3  EU27_2007_FOR    A  NO  EU27_2007_FOR   NR 